# Summarize microbiome data by genus

The goal of this notebook is to summarize microbiome data according to host taxonomy. 
This includes several measures of richness and evenness, as well as the most abundant microbial group at several levels of taxonomy. These are then merged with the disease data to produce a final trait table.

## Import necessary libraries

First we'll import needed libraries, taking some care to accomodate older versions of QIIME2


In [18]:
from qiime2 import Artifact
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.taxa.methods import filter_table,collapse

#The below try/except block is unsightly but the alpha function got moved between recent versions of QIIME2
#and it's nice if the notebook is compatible with either
try:
    from qiime2.plugins.diversity.methods import alpha
    from qiime2.plugins.diversity.methods import alpha_phylogenetic
except:
    from qiime2.plugins.diversity.pipelines import alpha
    from qiime2.plugins.diversity.pipelines import alpha_phylogenetic
    
from qiime2.plugins.diversity.visualizers import alpha_group_significance

from qiime2.plugins.feature_table.methods import rarefy
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.methods import filter_features

from qiime2.metadata import Metadata
from os.path import abspath,exists,join
from os import mkdir
import shutil

from IPython.core.display import HTML

import pandas as pd

## (Optional) Check what files are in the input directory

It can be useful to manually confirm that all the files are where we expect before we begin.

In [13]:
from os import listdir
listdir("../input")

['GCMP_EMP_map_r29.txt',
 'silva_metaxa2_reference_taxonomy.qza',
 'physeq_taxonomy.txt',
 'feature_table_decon_skeleton_1000.qza',
 'disease_by_genus.csv',
 'huang_roy_molecular.newick',
 'physeq_metadata.txt',
 'insertion-tree_silva_GCMP.qza',
 'feature_table_decon_tissue_1000.qza',
 'GCMP_EMP_map_r28_no_empty_samples.txt',
 'merged_disease_table.tsv',
 'feature_table_decon_all_1000.qza',
 'Book1.csv',
 'physeq_rooted_tree.qza',
 'huang_roy_molecular_r2.newick',
 'feature_table_decon_mucus_1000.qza',
 'merged_disease_table_r2_names_updated.txt',
 'physeq_feature_table.qza']

## Define paths for all required input files

We'll save the expected filename of each required input file to a variable. This lets us check that all are present before we do any work inside QIIME2.

In [24]:
#Set paths for all required files.
#Currently all are assumed to be directly in the ../input directory
#(if they are somewhere else, just modify the input_directory variable accordingly)
input_directory = abspath("../input")

mapping_file = join(input_directory,"GCMP_EMP_map_r28_no_empty_samples.txt")
taxonomy_file = join(input_directory,"silva_metaxa2_reference_taxonomy.qza") 
tree_file = join(input_directory,"physeq_rooted_tree.qza")
feature_table_all_path = join(input_directory,"feature_table_decon_all_1000.qza")
feature_table_mucus_path = join(input_directory,"feature_table_decon_mucus_1000.qza")
feature_table_tissue_path = join(input_directory,"feature_table_decon_tissue_1000.qza")
feature_table_skeleton_path = join(input_directory,"feature_table_decon_skeleton_1000.qza")

disease_data_path = join(input_directory,"disease_by_genus.csv")

## (Optional) Check that all required files are present

If any are missing, its easier to find out now, before we've spent a long time processing the data.

In [25]:
required_files = [mapping_file,taxonomy_file,tree_file,feature_table_all_path,feature_table_mucus_path,feature_table_tissue_path,\
  feature_table_skeleton_path,disease_data_path]

for required_filepath in required_files:
    
    if not exists(required_filepath):
        raise ValueError(f"Filepath {required_filepath} is needed for this workflow but isn't present. Could it be named something else or in another directory?")

    print(f"Filepath {required_filepath} exists and is accessible...OK")

Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/GCMP_EMP_map_r28_no_empty_samples.txt exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/silva_metaxa2_reference_taxonomy.qza exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/physeq_rooted_tree.qza exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/feature_table_decon_all_1000.qza exists and is accessible...OK
Filepath /Users/zaneveld/Dropbox/Zaneveld_Lab_Organization/Projects/GCMP_Global_Disease/gcmp_global_disease/analysis/core_analysis/input/feature_table_decon_mucus_1000.qza exists and is accessible...OK

## Create an output directory if one doesn't exist

In [26]:

output_dir = abspath("../output/")
ft_output_dir = join(output_dir,"feature_tables")

if not exists(ft_output_dir):
    print(f"Output directory {ft_output_dir} does not yet exist, creating it...")
    mkdir(ft_output_dir)
    print("Done.")

## Load feature tables, the coral tree, and metadata into QIIME2 as Artifacts

In [27]:

tree = Artifact.load(tree_file)
taxonomy = Artifact.load(taxonomy_file)
metadata = Metadata.load(mapping_file)
feature_table_decon_all_1000 = Artifact.load(feature_table_all_path)
feature_table_decon_mucus_1000 = Artifact.load(feature_table_mucus_path)
feature_table_decon_tissue_1000 = Artifact.load(feature_table_tissue_path)
feature_table_decon_skeleton_1000 = Artifact.load(feature_table_skeleton_path)

#create a dictionary to hold each feature table so they are easy to access in loops
feature_tables_decon_1000 = {"mucus":feature_table_decon_mucus_1000, "tissue":feature_table_decon_tissue_1000, "skeleton":feature_table_decon_skeleton_1000, "all":feature_table_decon_all_1000}




## Use the Metadata file to find a list of all unique coral genera in the study

In [28]:
# Find unique genus names
df = metadata.to_dataframe()
unique_species_names = list(set(list(df['host_genus'])))
print(f"There are {len(unique_species_names)} unique species names in the dataset:")
print(sorted(unique_species_names))

There are 74 unique species names in the dataset:
['Acanthastrea', 'Acropora', 'Aiptasia', 'Alveopora', 'Astrea', 'Aurelia', 'Caulastraea', 'Coscinaraea', 'Ctenactis', 'Cyphastrea', 'Danafungia', 'Dendrophyllia', 'Diploastrea', 'Diploria', 'Dipsastraea', 'Distichopora', 'Echinophyllia', 'Echinopora', 'Eguchipsammia', 'Entacmaea', 'Favites', 'Fungid', 'Galaxea', 'Gardineroseris', 'Goniastrea', 'Goniopora', 'Heliopora', 'Herpolitha', 'Heteractis', 'Homophyllia', 'Hydnophora', 'Isopora', 'Leptastrea', 'Leptoria', 'Lithophyllon', 'Lobophyllia', 'Lobophytum', 'Macrorhynchia', 'Merulina', 'Millepora', 'Missing: Not collected', 'Mnemiopsis', 'Montastraea', 'Montipora', 'Mycedium', 'Not applicable', 'Orbicella', 'Oxypora', 'Pachyseris', 'Palythoa', 'Pavona', 'Pectinia', 'Physogyra', 'Platygyra', 'Plerogyra', 'Pocillopora', 'Podabacia', 'Porites', 'Psammocora', 'Pseudosiderastrea', 'Rhodactis', 'Sarcophyton', 'Scolymia', 'Seriatopora', 'Siderastrea', 'Sinularia', 'Stephanocoenia', 'Stichodactyl

## Define a function for calculating the most abundant microbe in each host taxon

We want to know, for some level of taxonomic specificity, what the most abundant microbial group was in a particular taxon of hosts. For example, were Proteobacteria the most abundant phylum of microbes in all corals?

**Note: we don't call this function directly - it is instead called by the calculate per-species diversities function

In [43]:
 
#calculate the dominant taxon family
def get_dominant_taxon(feature_table,taxonomy,level=5):
    """Collapse the feature table at the specified level, then find which feature is most abundant"""
    try:
        collapse_results = collapse(feature_table,taxonomy,level)
        taxon_table = collapse_results.collapsed_table
        taxon_df = taxon_table.view(pd.DataFrame)
        #Calculate average abundance of each taxon in this species
    except TypeError:
        return None
    taxon_abundance_dict = {taxon_df[col].mean():col for col in list(taxon_df.columns)}
    
    mean_abundance = sorted(taxon_abundance_dict.keys())
    print(mean_abundance)
    if not mean_abundance:
        return None
    greatest_mean_abundance = mean_abundance[-1]
    most_abundant_taxon = taxon_abundance_dict[greatest_mean_abundance]
    print("Most abundant taxon:",most_abundant_taxon)
    return most_abundant_taxon
    
 

## Define a function for calculating alpha diversity within host taxa

We'll keep the specific column for the host taxon generic, so we can calculate per-species alpha diversity using e.g. the 'host_species' column, or per-genus diversity using 'host_genus' as the column.

In [44]:
def calculate_per_species_diversities(feature_table,\
                                      metadata,\
                                      species_column = "host_genus",\
                                      compartment_name = 'all',\
                                      taxonomy = None,\
                                      metrics = ['faith_pd','observed_features','gini_index','dominance','simpson_e'],\
                                      to_skip = ['none','','Not applicable','Missing: Not collected']\
                                      ):
    #Set up a DataFrame to hold results
    results_columns = [species_column,f"n_samples_{compartment_name}"]
    results_columns.extend([f"{metric}_{compartment_name}" for metric in metrics])
    taxonomy_levels = ('domain','phylum','class','order','family','genus')
    taxonomy_labels = [f"most_abundant_{level}_{compartment_name}" for level in taxonomy_levels]
    results_columns.extend(taxonomy_labels)
    print("Result columns:",results_columns)
    results_df = pd.DataFrame(columns = results_columns)
    results_df = results_df.set_index(species_column)
    metadata_df = metadata.to_dataframe()
    unique_species_names = list(set(list(metadata_df[species_column])))
    for species in unique_species_names:
        if species in to_skip:
            continue
            
        #Filter the feature table to just our current species
        where = f"[{species_column}] = '{species}'"
        filter_results = filter_samples(feature_table, metadata = metadata,where = where)
        species_table = filter_results.filtered_table
        #We'll use the species table, not the overall feature table from here on down!
        
        print("Analyzing taxon:",species)
        #If taxonomy is provided, summarize the type of microbe with highest average
        #abundance at each taxonomic level
        if taxonomy:
            for i, taxon_label in enumerate(taxonomy_levels):
                level = i + 1 #domain is level 1 in QIIME2, not level 0
                most_abundant_taxon = get_dominant_taxon(species_table,taxonomy,level=level)
                column_label = f"most_abundant_{taxon_label}_{compartment_name}"
                results_df.loc[species,column_label] = most_abundant_taxon
            
        
        for metric in metrics:
            #print(f"Calculating {metric} for {species}")
            try:
                if metric == "faith_pd":
                    alpha_results = alpha_phylogenetic(species_table,phylogeny=tree,metric=metric)
                else:
                    alpha_results = alpha(species_table, metric = metric)
            except ValueError:
                print(f"Can't calculate {metric} for {species} {compartment_name}...skipping")
                continue
            
            alpha_diversity = alpha_results.alpha_diversity
            species_adiv = alpha_diversity.view(pd.Series)
            
            species_mean = species_adiv.mean()
            results_df.loc[species,f"{metric}_{compartment_name}"] = species_mean
            print(f"{species}\t{metric}\t{compartment_name}\t{round(species_mean,4)}")
            
            
            
        #For last metric only we'll grab n (should be the same for all)
        species_n = len(species_adiv)     
        results_df.loc[species,f"n_samples_{compartment_name}"]=species_n
    return results_df

## Generate microbial results .tsv files for each of our species 

In [45]:
   
results_dfs = {} 
for compartment,table in sorted(feature_tables_decon_1000.items(),reverse=True):
    result_df = calculate_per_species_diversities(table,metadata,compartment_name = compartment,taxonomy = taxonomy)
    species_adiv_path = join(output_dir,f"adiv_trait_table_{compartment}.tsv")
    results_dfs[compartment]=result_df
    result_df.to_csv(species_adiv_path,sep="\t")

Result columns: ['host_genus', 'n_samples_tissue', 'faith_pd_tissue', 'observed_features_tissue', 'gini_index_tissue', 'dominance_tissue', 'simpson_e_tissue', 'most_abundant_domain_tissue', 'most_abundant_phylum_tissue', 'most_abundant_class_tissue', 'most_abundant_order_tissue', 'most_abundant_family_tissue', 'most_abundant_genus_tissue']
Analyzing taxon: Leptastrea
[77.0, 256.0, 667.0]
Most abundant taxon: D_0__Bacteria
[1.0, 2.0, 4.0, 5.0, 7.0, 13.0, 15.0, 23.0, 38.0, 77.0, 99.0, 102.0, 105.0, 114.0, 161.0, 218.0]
Most abundant taxon: D_0__Archaea;D_1__Thaumarchaeota
[1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 13.0, 15.0, 17.0, 35.0, 38.0, 62.0, 67.0, 77.0, 98.0, 114.0, 157.0, 218.0]
Most abundant taxon: D_0__Archaea;D_1__Thaumarchaeota;D_2__Nitrososphaeria
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 10.0, 11.0, 13.0, 14.0, 17.0, 18.0, 28.0, 37.0, 49.0, 51.0, 53.0, 56.0, 77.0, 86.0, 99.0, 218.0]
Most abundant taxon: D_0__Archaea;D_1__Thaumarchaeota;D_2__Nitrososphaeria;D_3__Nitrosopumilales
[1.0, 2.0, 3

Pavona	faith_pd	tissue	100.4108
Pavona	observed_features	tissue	268.7083
Pavona	gini_index	tissue	0.9731
Pavona	dominance	tissue	0.0507
Pavona	simpson_e	tissue	0.1517
Analyzing taxon: Aurelia
[1.5, 998.5]
Most abundant taxon: D_0__Bacteria
[0.5, 1.5, 3.5, 10.0, 14.0, 17.5, 36.5, 250.0, 666.0]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
[0.5, 1.0, 1.5, 3.5, 4.5, 5.5, 14.0, 17.5, 36.5, 221.5, 250.0, 443.5]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
[0.5, 1.0, 1.5, 3.5, 4.0, 4.5, 11.0, 12.0, 13.5, 17.5, 25.5, 90.0, 119.0, 201.5, 242.0, 250.0]
Most abundant taxon: D_0__Bacteria;D_1__Tenericutes;D_2__Mollicutes;D_3__Mycoplasmatales
[0.5, 1.0, 1.5, 3.0, 3.5, 4.5, 5.5, 10.0, 11.5, 13.5, 14.5, 17.5, 37.0, 90.0, 119.0, 201.5, 205.0, 250.0]
Most abundant taxon: D_0__Bacteria;D_1__Tenericutes;D_2__Mollicutes;D_3__Mycoplasmatales;D_4__Mycoplasmataceae
[0.5, 1.0, 1.5, 2.0, 3.0, 3.5, 4.5, 5.5, 10.0, 11.5, 13.5, 14.5, 17.0, 17.5, 18.0, 37.0, 71.0, 119.0

Acanthastrea	faith_pd	tissue	62.6728
Acanthastrea	observed_features	tissue	148.0
Acanthastrea	gini_index	tissue	0.9199
Acanthastrea	dominance	tissue	0.0662
Acanthastrea	simpson_e	tissue	0.1913
Analyzing taxon: Lobophytum
[5.0, 995.0]
Most abundant taxon: D_0__Bacteria
[5.0, 8.0, 34.0, 36.0, 169.0, 704.0]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
[5.0, 8.0, 34.0, 36.0, 41.0, 128.0, 318.0, 386.0]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria
[1.0, 5.0, 8.0, 28.0, 30.0, 34.0, 36.0, 41.0, 61.0, 98.0, 159.0, 215.0, 226.0]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Betaproteobacteriales
[1.0, 2.0, 3.0, 5.0, 8.0, 14.0, 15.0, 16.0, 17.0, 24.0, 25.0, 28.0, 36.0, 38.0, 41.0, 61.0, 66.0, 159.0, 160.0, 215.0]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Sphingomonadales;D_4__Sphingomonadaceae
[1.0, 2.0, 3.0, 5.0, 8.0, 10.0, 14.0, 15.0, 16.0, 17.0, 24.0, 25.0, 28.0, 36.0,

[0.3333333333333333, 0.6666666666666666, 1.3333333333333333, 1.6666666666666667, 2.3333333333333335, 4.333333333333333, 7.333333333333333, 11.0, 118.0, 320.3333333333333, 526.0]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
[0.3333333333333333, 0.6666666666666666, 1.0, 1.3333333333333333, 1.6666666666666667, 2.0, 2.3333333333333335, 2.6666666666666665, 3.6666666666666665, 4.333333333333333, 4.666666666666667, 7.333333333333333, 7.666666666666667, 11.0, 115.0, 303.0, 517.6666666666666]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodospirillales
[0.3333333333333333, 0.6666666666666666, 1.0, 1.3333333333333333, 1.6666666666666667, 2.3333333333333335, 2.6666666666666665, 3.6666666666666665, 4.333333333333333, 4.666666666666667, 7.333333333333333, 11.0, 113.66666666666667, 302.6666666666667, 517.6666666666666]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhodospirillales;D_4__

[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.5, 5.75, 6.0, 6.25, 7.25, 8.25, 8.75, 9.25, 10.75, 11.5, 11.75, 12.25, 13.25, 14.0, 15.25, 15.5, 15.75, 20.0, 26.5, 29.25, 31.5, 37.75, 39.75, 42.25, 42.75, 53.25, 63.75, 70.5, 72.0]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Sphingomonadales;D_4__Sphingomonadaceae
[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.25, 5.5, 5.75, 6.0, 6.25, 7.0, 7.25, 8.75, 10.5, 10.75, 11.25, 11.75, 12.75, 13.25, 15.25, 15.5, 15.75, 18.25, 19.5, 20.0, 24.0, 29.25, 29.5, 31.5, 34.5, 39.75, 42.25, 48.5, 50.75, 51.25]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rhizobiales;D_4__Beijerinckiaceae;D_5__Methylobacterium
Gardineroseris	faith_pd	tissue	94.5683
Gardineroseris	observed_features	tissue	261.25
Gardineroseris	gini_index	tissue	0.8586
Gardineroseris	domina

...

[0.034482758620689655, 0.06896551724137931, 0.10344827586206896, 0.13793103448275862, 0.1724137931034483, 0.20689655172413793, 0.2413793103448276, 0.27586206896551724, 0.3103448275862069, 0.3448275862068966, 0.3793103448275862, 0.41379310344827586, 0.4482758620689655, 0.4827586206896552, 0.5172413793103449, 0.5517241379310345, 0.5862068965517241, 0.6206896551724138, 0.6551724137931034, 0.6896551724137931, 0.7241379310344828, 0.7586206896551724, 0.7931034482758621, 0.8275862068965517, 0.8620689655172413, 0.896551724137931, 0.9310344827586207, 0.9655172413793104, 1.0, 1.0344827586206897, 1.0689655172413792, 1.103448275862069, 1.1724137931034482, 1.206896551724138, 1.2413793103448276, 1.2758620689655173, 1.3103448275862069, 1.3448275862068966, 1.3793103448275863, 1.4137931034482758, 1.4482758620689655, 1.4827586206896552, 1.5172413793103448, 1.5517241379310345, 1.5862068965517242, 1.7241379310344827, 1.793103448275862, 1.8620689655172413, 1.896551724137931, 1.9310344827586208, 2.068965517

[0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.2, 4.8, 5.0, 5.2, 5.4, 5.8, 6.0, 6.4, 7.0, 7.2, 7.8, 8.6, 9.0, 9.2, 11.0, 11.6, 12.2, 12.4, 13.0, 14.6, 15.2, 17.4, 21.0, 24.2, 28.8, 31.6, 34.4, 35.2, 36.0, 45.8, 48.4, 55.4, 96.6]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Dstr-E11;D_4__uncultured alpha proteobacterium;D_5__uncultured alpha proteobacterium
Physogyra	faith_pd	skeleton	94.8124
Physogyra	observed_features	skeleton	254.4
Physogyra	gini_index	skeleton	0.8923
Physogyra	dominance	skeleton	0.058
Physogyra	simpson_e	skeleton	0.203
Analyzing taxon: Heteractis
[]
[]
[]
[]
[]
[]
Can't calculate faith_pd for Heteractis skeleton...skipping
Can't calculate observed_features for Heteractis skeleton...skipping
Can't calculate gini_index for Heteractis skeleton...skipping
Can't calculate dominance for Heteractis skeleton...skipping
Can't calculate simpson_e for Heteractis skeleton...skipping
Analy

Pocillopora	faith_pd	skeleton	39.8848
Pocillopora	observed_features	skeleton	83.9167
Pocillopora	gini_index	skeleton	0.9825
Pocillopora	dominance	skeleton	0.2187
Pocillopora	simpson_e	skeleton	0.1273
Analyzing taxon: Homophyllia
[0.6666666666666666, 85.0, 914.3333333333334]
Most abundant taxon: D_0__Bacteria
[0.3333333333333333, 0.6666666666666666, 2.0, 3.3333333333333335, 5.0, 5.333333333333333, 6.0, 7.666666666666667, 13.666666666666666, 17.666666666666668, 60.333333333333336, 85.0, 89.33333333333333, 177.66666666666666, 524.3333333333334]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria
[0.3333333333333333, 0.6666666666666666, 1.3333333333333333, 2.6666666666666665, 3.3333333333333335, 5.0, 5.333333333333333, 6.0, 7.666666666666667, 8.0, 13.666666666666666, 15.0, 15.333333333333334, 21.0, 34.333333333333336, 45.0, 47.0, 85.0, 161.66666666666666, 177.0, 341.3333333333333]
Most abundant taxon: D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria
[0.3333333333333333, 0.6666

[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 5.0, 5.25, 5.5, 6.0, 6.25, 6.75, 8.25, 8.5, 9.5, 10.0, 10.5, 11.0, 11.75, 12.0, 13.5, 14.25, 16.0, 16.5, 17.75, 19.0, 20.0, 31.0, 31.25, 32.25, 38.0, 58.0, 62.0, 86.75, 141.0]
Most abundant taxon: D_0__Bacteria;__;__;__;__;__
Scolymia	faith_pd	skeleton	76.2054
Scolymia	observed_features	skeleton	193.5
Scolymia	gini_index	skeleton	0.895
Scolymia	dominance	skeleton	0.121
Scolymia	simpson_e	skeleton	0.1532
Analyzing taxon: Stichodactyla
[]
[]
[]
[]
[]
[]
Can't calculate faith_pd for Stichodactyla skeleton...skipping
Can't calculate observed_features for Stichodactyla skeleton...skipping
Can't calculate gini_index for Stichodactyla skeleton...skipping
Can't calculate dominance for Stichodactyla skeleton...skipping
Can't calculate simpson_e for Stichodactyla skeleton...skipping
Analyzing taxon: Podabacia
[1.5, 11.5, 987.0]
Most abundant taxon: D_0__Bacteria
[0.5, 1.0, 1.5, 2.5, 3.5, 6.5, 10.0

UnboundLocalError: local variable 'species_adiv' referenced before assignment

## Integrate these new microbial data with existing disease data

In [46]:
#Integrate data into a per species level table
from IPython.core.display import HTML,display

disease_df = pd.read_csv("../input/disease_by_genus.csv")
disease_df = disease_df.set_index('host_genus')
for compartment,results_df in results_dfs.items():
    results_df.index.rename('host_genus')    
    disease_df = pd.merge(disease_df, results_df, how="outer", left_index = True, right_index = True, indicator=False)
disease_df.to_csv("../output/GCMP_trait_table_genus.tsv",sep="\t")
